# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [70]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [20]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('Weather_Database/WeatherPy_Database.csv', index_col=0)
# Original file was saved without index=False

# Display sample data

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,78.62,77,98,14.23,light rain
1,Uyovu,TZ,-3.2833,31.5258,63.77,88,84,1.39,light rain
2,New Norfolk,AU,-42.7826,147.0587,64.90,53,56,5.01,broken clouds
3,Hobart,AU,-42.8794,147.3294,65.26,82,75,13.80,broken clouds
4,Aitape,PG,-3.1374,142.3475,85.41,68,71,8.55,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [30]:
# Prompt the user to enter minimum and maximum temperature criteria.

min_temp = float(input("What is the minimum temperature (°F) you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature (°F) you would like for your vacation? "))


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [34]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]


# Display sample data
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Uyovu,TZ,-3.2833,31.5258,63.77,88,84,1.39,light rain
2,New Norfolk,AU,-42.7826,147.0587,64.90,53,56,5.01,broken clouds
3,Hobart,AU,-42.8794,147.3294,65.26,82,75,13.80,broken clouds
12,Castro,BR,-24.7911,-50.0119,55.31,96,100,3.87,overcast clouds
20,Bredasdorp,ZA,-34.5322,20.0403,60.22,61,100,10.16,overcast clouds
21,Abu Dhabi,AE,24.4667,54.3667,68.11,62,0,4.61,clear sky
27,Jamestown,US,42.0970,-79.2353,50.02,71,100,17.27,overcast clouds
28,East London,ZA,-33.0153,27.9116,66.88,90,100,9.17,moderate rain
30,Alvinopolis,BR,-20.1067,-43.0489,68.29,97,100,3.11,overcast clouds
32,Cape Town,ZA,-33.9258,18.4232,61.21,73,75,19.57,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [52]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(axis=0,how="any")
# Display sample data

preferred_cities_df.count()

City                   178
Country                171
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [53]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [54]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Uyovu,TZ,63.77,light rain,-3.2833,31.5258,
2,New Norfolk,AU,64.90,broken clouds,-42.7826,147.0587,
3,Hobart,AU,65.26,broken clouds,-42.8794,147.3294,
12,Castro,BR,55.31,overcast clouds,-24.7911,-50.0119,
20,Bredasdorp,ZA,60.22,overcast clouds,-34.5322,20.0403,
21,Abu Dhabi,AE,68.11,clear sky,24.4667,54.3667,
27,Jamestown,US,50.02,overcast clouds,42.0970,-79.2353,
28,East London,ZA,66.88,moderate rain,-33.0153,27.9116,
30,Alvinopolis,BR,68.29,overcast clouds,-20.1067,-43.0489,
32,Cape Town,ZA,61.21,broken clouds,-33.9258,18.4232,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [55]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [57]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Uyovu - nearest hotel: No hotel found
New Norfolk - nearest hotel: Woodbridge on the Derwent
Hobart - nearest hotel: Vibe Hotel Hobart
Castro - nearest hotel: Hotel Central Palace
Bredasdorp - nearest hotel: Victoria Hotel
Abu Dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
Jamestown - nearest hotel: DoubleTree Jamestown
East London - nearest hotel: No hotel found
Alvinopolis - nearest hotel: Hotel Nardy
Cape Town - nearest hotel: Townhouse Hotel
Kamariotissa - nearest hotel: Atzanou rooms
Guerrero Negro - nearest hotel: Plaza sal paraiso
Plettenberg Bay - nearest hotel: Milkwood Manor
Hermanus - nearest hotel: Aloe guest house
Ledyard - nearest hotel: No hotel found
Grootfontein - nearest hotel: Meteor Travel Inn
Te Anau - nearest hotel: Lakeside Motels & Apartments
Camacha - nearest hotel: Praia Dourada
Laguna - nearest hotel: Holiday Inn Express & Suites
Ribeira Grande - nearest hotel: No hotel found
Najran - nearest hotel: No hotel found
Margate - nearest h

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Uyovu,TZ,63.77,light rain,-3.2833,31.5258,No hotel found
2,New Norfolk,AU,64.90,broken clouds,-42.7826,147.0587,Woodbridge on the Derwent
3,Hobart,AU,65.26,broken clouds,-42.8794,147.3294,Vibe Hotel Hobart
12,Castro,BR,55.31,overcast clouds,-24.7911,-50.0119,Hotel Central Palace
20,Bredasdorp,ZA,60.22,overcast clouds,-34.5322,20.0403,Victoria Hotel
...,...,...,...,...,...,...,...
645,Moussoro,TD,62.29,clear sky,13.6408,16.4901,No hotel found
651,Chor,PK,51.33,clear sky,25.5167,69.7667,No hotel found
656,Tenenkou,ML,64.31,few clouds,14.4572,-4.9169,No hotel found
657,Ixtapa,MX,67.24,clear sky,20.7000,-105.2000,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [66]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='No hotel found')]

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,New Norfolk,AU,64.90,broken clouds,-42.7826,147.0587,Woodbridge on the Derwent
3,Hobart,AU,65.26,broken clouds,-42.8794,147.3294,Vibe Hotel Hobart
12,Castro,BR,55.31,overcast clouds,-24.7911,-50.0119,Hotel Central Palace
20,Bredasdorp,ZA,60.22,overcast clouds,-34.5322,20.0403,Victoria Hotel
21,Abu Dhabi,AE,68.11,clear sky,24.4667,54.3667,Marriott Downtown Abu-Dhabi
...,...,...,...,...,...,...,...
620,Porto Belo,BR,67.95,scattered clouds,-27.1578,-48.5531,Residencial Maravilha
623,Rabo De Peixe,PT,60.31,light rain,37.8000,-25.5833,Pico do Refugio
632,Galveston,US,60.94,clear sky,29.3669,-94.9669,Comfort Inn & Suites Texas City
636,Imbituba,BR,65.46,few clouds,-28.2400,-48.6703,Silvestre Praia Hotel


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [62]:
# Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file,index = False)

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [73]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)